In [54]:
import sys
import os
sys.path.append('../')
import pathlib as pl
from SymEigen import *
from sympy import symbols
from project_dir import project_dir
Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")


In [55]:
x = Eigen.Vector("x", 3)
P = Eigen.Vector("P", 3)
N = Eigen.Vector("N", 3)

Cl0 = Gen.Closure(x, P, N)

D = ((x-P).dot(N))**2
D

(N(0)*(-P(0) + x(0)) + N(1)*(-P(1) + x(1)) + N(2)*(-P(2) + x(2)))**2

In [56]:
dDdx = VecDiff(D, x)
dDdx

Matrix([
[2*N(0)*(N(0)*(-P(0) + x(0)) + N(1)*(-P(1) + x(1)) + N(2)*(-P(2) + x(2)))],
[2*N(1)*(N(0)*(-P(0) + x(0)) + N(1)*(-P(1) + x(1)) + N(2)*(-P(2) + x(2)))],
[2*N(2)*(N(0)*(-P(0) + x(0)) + N(1)*(-P(1) + x(1)) + N(2)*(-P(2) + x(2)))]])

In [57]:
ddDddx = VecDiff(dDdx, x)
ddDddx

Matrix([
[  2*N(0)**2, 2*N(0)*N(1), 2*N(0)*N(2)],
[2*N(0)*N(1),   2*N(1)**2, 2*N(1)*N(2)],
[2*N(0)*N(2), 2*N(1)*N(2),   2*N(2)**2]])

In [58]:
x0 = Eigen.Vector("x_prev", 3)
e1 = Eigen.Vector("e1", 3)
e2 = Eigen.Vector("e2", 3)

dx = x - x0
dx

Matrix([
[x(0) - x0(0)],
[x(1) - x0(1)],
[x(2) - x0(2)]])

In [59]:
u = dx.dot(e1)
v = dx.dot(e2)

TR = Matrix([[u], [v]])
TR

Matrix([
[e1(0)*(x(0) - x0(0)) + e1(1)*(x(1) - x0(1)) + e1(2)*(x(2) - x0(2))],
[e2(0)*(x(0) - x0(0)) + e2(1)*(x(1) - x0(1)) + e2(2)*(x(2) - x0(2))]])

In [60]:
dTRdx = VecDiff(TR, x)
dTRdx

Matrix([
[e1(0), e1(1), e1(2)],
[e2(0), e2(1), e2(2)]])

In [61]:
Cl1 = Gen.Closure(x, x0, e1, e2)

In [62]:
s = f'''
// > Squared Version
// > D := d*d 
// > DHat := dHat*dHat

{Cl0("HalfPlaneD",D)}
{Cl0("dHalfPlaneDdx",dDdx)}
{Cl0("ddHalfPlaneDddx",ddDddx)}
{Cl1("TR",TR)}
{Cl1("dTRdx",dTRdx)}
'''
print(s)

f = open( project_dir() / 'src/uipc/backends/cuda/contact_system/contact_models/sym/vertex_half_plane_distance.inl', 'w')
f.write(s)
f.close()


// > Squared Version
// > D := d*d 
// > DHat := dHat*dHat

template <typename T>
__host__ __device__ void HalfPlaneD(T& R, const Eigen::Vector<T,3>& x, const Eigen::Vector<T,3>& P, const Eigen::Vector<T,3>& N)
{
/*****************************************************************************************************************************
Function generated by SymEigen.py 
Author: MuGdxy
GitHub: https://github.com/MuGdxy/SymEigen
E-Mail: lxy819469559@gmail.com
******************************************************************************************************************************
LaTeX expression:
//tex:$$R = \left(N(0) \left(- P(0) + x(0)\right) + N(1) \left(- P(1) + x(1)\right) + N(2) \left(- P(2) + x(2)\right)\right)^{2}$$

Symbol Name Mapping:
x:
    -> {}
    -> Matrix([[x(0)], [x(1)], [x(2)]])
P:
    -> {}
    -> Matrix([[P(0)], [P(1)], [P(2)]])
N:
    -> {}
    -> Matrix([[N(0)], [N(1)], [N(2)]])
******************************************************************************